In [ ]:
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen

file=urlopen('https://raw.githubusercontent.com/ellikamishra/Datasets/master/MitreData.json')
# file = open('MitreData.json')

obj_list = []
dict = {}

for line in file:
    # line=line + '\n'
    obj_list.append(eval(line))

new_list = []
for item in obj_list:
    # if(len(item['tactics'])>1):
    #     for tactic in item['tactics']:
    #         dict = item
    #         dict['tactics'] = tactic
    #         new_list.append(dict)
    # else:
    dict = item
    #dict['tactics'] = item['tactics'][0]
    new_list.append(dict)

df = pd.DataFrame(new_list)
df.head()

In [ ]:
fileD = urlopen('https://raw.githubusercontent.com/ellikamishra/Datasets/master/DataSources.json')

objD_list = []
for line in fileD:
    objD_list.append(eval(line))
    
df_DS = pd.DataFrame(objD_list)
df_DS.head()

In [ ]:
tramp_small=pd.read_json('https://raw.githubusercontent.com/center-for-threat-informed-defense/tram/master/tests/data/test-training-data.json')

tramp=pd.read_json('https://raw.githubusercontent.com/center-for-threat-informed-defense/tram/master/data/training/bootstrap-training-data.json')

df_tramp=tramp['sentences']
df_tramp_small=tramp_small['sentences']

df_tramp.append(df_tramp_small,ignore_index=True)



# print(df_tramp[200])

In [ ]:


dict_tr={'id':[],'text':[]}
index=0
for i in df_tramp:
    # print(i['mappings'][0]['attack_id'])
    # print(i)
    if not i['mappings']:
        continue
    idv=i['mappings'][0]['attack_id']
    
    dict_tr['id'].append(idv)
    dict_tr['text'].append(i['text'])
    
df_tr=pd.DataFrame(dict_tr)

In [ ]:
df_comb=df.join(df_tr.set_index('id'),on='id')

In [ ]:
# df_comb.drop(df.iloc[:, 8:22], inplace = True, axis = 1)
df_comb.drop(['subtechniqueof'],axis=1)
df_comb.drop(['subtechniques'],axis=1)
# df_comb=df_comb.fillna(df_comb.mode().iloc[0])
# df_comb.head()
for x in df_comb:
    # print(df_comb[x])
    df_comb[x]=df_comb[x].fillna(method='bfill',axis=0)
    df_comb[x]=df_comb[x].fillna(method='ffill',axis=0)
    df_comb[x]=df_comb[x].interpolate(method='linear',limit_direction='forward',axis=0)

df_comb.isnull().sum()

In [ ]:
df_comb.drop(['subtechniqueof'],axis=1)
df_comb.drop(['subtechniques'],axis=1)
# df_comb=df_comb.fillna(df_comb.mode().iloc[0])
# df_comb.head()
for x in df_comb:
    # print(df_comb[x])
    df_comb[x]=df_comb[x].fillna(method='bfill',axis=0)
    df_comb[x]=df_comb[x].fillna(method='ffill',axis=0)
    df_comb[x]=df_comb[x].interpolate(method='linear',limit_direction='forward',axis=0)

df_comb.isnull().sum()

In [ ]:
df_comb['id'].value_counts()
tids = df_comb['id'].unique()
dict_target = {}
dict_tactic= {}
i = 0
for x in tids:
     dict_target[x] = i
     dict_tactic[x]=df_comb.loc[df_comb['id']==x,'tactics'].append(df_comb.loc[df_comb['id']==x,'techniquename'])
     i = i + 1
#print(dict_target)

df_comb['num_target']=df_comb['id'].map(dict_target)

In [ ]:
from scipy.stats import sem
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold,StratifiedKFold,StratifiedShuffleSplit,StratifiedGroupKFold,RepeatedStratifiedKFold, GroupShuffleSplit,ShuffleSplit
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score,cross_val_predict

X = df_comb.loc[:, df_comb.columns != 'techniquename'].apply(lambda x: ",".join(x.astype(str)), axis=1)
y=df_comb['num_target']


vect = CountVectorizer(ngram_range=(2,2),stop_words='english')

vect.fit_transform(X)
# vect.transform(y)
model = MultinomialNB(alpha =0.2)

clf=make_pipeline(vect,model)

cv = RepeatedKFold(n_splits=4, n_repeats=10, random_state=1)
# 0.75257732 0.73966942 0.76033058 0.76239669
cvr=RepeatedStratifiedKFold(n_splits=4,n_repeats=10,random_state=1)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
# [0.75       0.75773196 0.7622739  0.75452196 0.76744186]

sfold=ShuffleSplit(n_splits=6, test_size=0.4, random_state=1)
# [0.74580645 0.74967742 0.73935484 0.72645161 0.73419355 0.7483871 ]



In [ ]:
for train_id,test_id in kfold.split(X,y):
  # X_train,X_test=X[train_id],X[test_id]
  clf.fit(X.iloc[train_id],y.iloc[train_id])
  clf.predict(X.iloc[test_id])

In [ ]:
def predict_id(num):
    #print(dict_target)
    for key, value in dict_target.items():
        if num == value:
            return key

In [ ]:
#T1558.004 AS-REP Roasting
desc = ["Monitor network traffic for unusual ARP traffic, gratuitous ARP replies may be suspicious. Consider collecting changes to ARP caches across endpoints for signs of ARP poisoning. For example, if multiple IP addresses map to a single MAC address, this could be an indicator that the ARP cache has been poisoned"] 
# T1134 Access Token Manipulation
desc3=["Monitor the file system for files that have the setuid or setgid bits set. Also look for any process API calls for behavior that may be indicative of [Process Injection]"]
desc2 = ["Suspicious Process Activity- Targeted - Malicious Start Menu Startup Modification Analytic -A2B - EDR "]
# test = vect.transform(desc3)
# pred=cross_val_predict(clf, X, y, cv=cvr)

t1=["Service providers have significant access to customer networks, enabling an attacker who had compromised a service provider to move laterally into the network of the service provider customer "]
pred= clf.predict(t1)
print(pred)


r1=predict_id(pred)
r = dict_tactic[r1]
print (r)
print(r1)